In [1]:
import pandas as pd
import yaml

# 1. Read given files

1.1 Read YAML

In [2]:
with open('test_settings.yaml', 'r') as file:
    settings = yaml.safe_load(file)

mass_storage_liquid = settings['storage_liquid']['mass_storage_liquid_in_g'] # access nested dictionary value
lot_data = settings['storage_liquid']['storage_liquid_lot_number'] # access nested dictionary value

storage_liquids_list = [
    {
        'lot_number': lot_number,
        'mass_storage_liquid_in_g': mass_storage_liquid,
        'density_storage_liquid_in_g_per_mL': lot_info['density_storage_liquid_in_g_per_mL']
    }
    for lot_number, lot_info in lot_data.items()
]

storage_liquids = pd.DataFrame(storage_liquids_list)

storage_liquids

,lot_number,mass_storage_liquid_in_g,density_storage_liquid_in_g_per_mL
0,00039,7.01,0.980
1,00040,7.01,1.003
2,223456789,7.01,1.010


1.2 Read CSV

In [6]:
metadata = pd.read_csv('test_metadata.csv', dtype={'storage_liquid_lot_number': str})
metadata

,sample_name,num_ms,storage_liquid_lot_number,col_phenotype
0,SEPT19,16.9,00039,blue
1,04409,15.9,00039,green
2,rathbone,40.0,00040,green
3,04409,15.2,223456789,green
4,11855,42.0,00039,blue
5,SHR93,16.1,223456789,green


# 2. Merge Data and Calculate Volume

2.1 Merge the two dataframes, **`storage_liquids`** from `test_settings.yaml` and **`metadata`** from `test_metadata.csv`.

In [7]:
# merge the two dataframes
merged_data = metadata.merge(storage_liquids, left_on='storage_liquid_lot_number', right_on='lot_number')
merged_data

,sample_name,num_ms,storage_liquid_lot_number,col_phenotype,lot_number,mass_storage_liquid_in_g,density_storage_liquid_in_g_per_mL
0,SEPT19,16.9,00039,blue,00039,7.01,0.980
1,04409,15.9,00039,green,00039,7.01,0.980
2,11855,42.0,00039,blue,00039,7.01,0.980
3,rathbone,40.0,00040,green,00040,7.01,1.003
4,04409,15.2,223456789,green,223456789,7.01,1.010
5,SHR93,16.1,223456789,green,223456789,7.01,1.010


2.2 Calculate the volume (in L) & add it into the new column

In [8]:
# calculate volumn & save into new columns
merged_data['volume_storage_liquid_in_L'] = (merged_data['mass_storage_liquid_in_g'] / merged_data['density_storage_liquid_in_g_per_mL']) * 0.001

extended_metadata = merged_data # optional: named 'extended_metadata' just to be clear the dataframe has been extended
extended_metadata

,sample_name,num_ms,storage_liquid_lot_number,col_phenotype,lot_number,mass_storage_liquid_in_g,density_storage_liquid_in_g_per_mL,volume_storage_liquid_in_L
0,SEPT19,16.9,00039,blue,00039,7.01,0.980,0.007153
1,04409,15.9,00039,green,00039,7.01,0.980,0.007153
2,11855,42.0,00039,blue,00039,7.01,0.980,0.007153
3,rathbone,40.0,00040,green,00040,7.01,1.003,0.006989
4,04409,15.2,223456789,green,223456789,7.01,1.010,0.006941
5,SHR93,16.1,223456789,green,223456789,7.01,1.010,0.006941


# 3. Return as TSV file

In [9]:
extended_metadata.to_csv('extended_metadata.tsv', sep='\t', index=False)

Links to the references:

- https://www.geeksforgeeks.org/parse-a-yaml-file-in-python/
- https://www.geeksforgeeks.org/how-to-write-pandas-dataframe-as-tsv-using-python/
